# 4.0 Modeling 

Contents

4.1 [Introduction](#4.1-Introduction)

  * [4.1.1 Problem Recap](#4.1.1-Problem-Recap)
  * [4.1.2 Notebook Goals](#4.1.2-Notebook-Goals)
 
4.2 [Load the data](#4.2-Load-the-data)

  * [4.2.1 Imports](#4.2.1-Imports)
  * [4.2.2 Load dataframe](#4.2.2-Load-dataframe)

4.3 [Vectorize and split data](#4.3-Vectorize-and-split-data)

4.4 [Sampling Techniques for Imbalance](#4.4-Sampling-Techniques-for-Imbalance)

  * [4.4.1 Sampling overview](#4.4.1-sampling-overview)
  * [4.4.2 Timing Comparisons](#4.4.2-timing-comparison-of-samplers)

4.5 [Bayesian Hyperparameter Optimization with Optuna](#4.5-bayesian-hyperparameter-optimization-with-optuna)

  * [4.5.1 About Optuna](#4.5.1-about-optuna:)
  * [4.5.2 Custom Optimization Function](#4.5.2-custom-optimization-function)
  * [4.5.3 Run 1: Small training size](#4.5.3-run-1)
  * [4.5.4 Run 2 Medium training size](#4.5.4-run-2)
  * [4.5.5 Run 3 Large training size, no sampling](#4.5.5-run-3)

4.6 [Modeling Summary](#4.6-modeling-summary)

  * [4.6.1 Model Comparisons](#461-model-comparisons)
  * [4.6.2 Conclusions](#462-conclusion)

## 4.1 Introduction

### 4.1.1 Problem Recap

Using customer text data about amazon products, we will build, evaluate and compare models to estimate the probability that a given text review can be classified as “positive” or “negative”.

Our goal is to build a text classifier using Amazon product review data which can be used to analyze customer sentiment which does not have accompanying numeric data. The metric we will be primarily interested in will be Recall on the positive class. This is the proportion of the positive class (negative reviews coded as "1" in the data) we correctly predict.

### 4.1.2 Notebook Goals

1. In our previous notebook our best results came from Term-Frequency Inverse-Document Frequency vectorization and a Logistic Regression Model.

2. We had slightly worse results from a Naive Bayes and Random Forest model. The Naive Bayes model incorrectly predicted a higher proportion of the negative class and the Random Forest model appeared to strongly overfit the training data with a very poor Recall on the test set.

3. Try over-sampling the minority class that we are trying to predict (encoded as "1"s) and/or under-sampling the majority class.

4. Test some other models such as gradient boosted trees (LightGBM/XGBoost) 

5. Examine how well our models will generalize with random sub-samples of the data.

6. Tune hyperparameters with bayesian search optimization using [Optuna](https://optuna.org/).

## 4.2 Load the data

### 4.2.1 Imports

In [1]:
from random import seed

#reading/processing data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import plotly.express as px

#splitting the dataset
from sklearn.model_selection import train_test_split

#scaling/vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

# models
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from imblearn.pipeline import Pipeline
import lightgbm as lgb

#metrics
from sklearn.metrics import recall_score

#dealing with class imbalance
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

#hyperparameter tuning
import optuna


/home/paul/anaconda3/envs/pyspark/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### 4.2.2 Load dataframe

In [2]:
data = pq.read_table("../data/edited/fashion.parquet")
fashion = data.to_pandas()
fashion

,review,neg_sentiment,stars,review_length
0,exactly needed,0,5,4
1,agree review opening small bent hook expensiv...,1,2,49
2,love going order pack work including losing ea...,0,4,50
3,tiny opening,1,2,4
4,okay,1,3,1
...,...,...,...,...
883631,absolutely love dress sexy comfortable split ...,0,5,51
883632,lbs tall wear large ordered large comfortable...,0,5,39
883633,big chest area,1,3,6
883634,clear needs lining,1,3,7


## 4.3 Vectorize and split data

In [3]:
#Vectorizing and splitting the data into train and test sets

tfidf = TfidfVectorizer(ngram_range=(1,2), min_df = 5, max_df=0.95)

X_train, X_test, y_train, y_test = train_test_split(fashion["review"].values, fashion["neg_sentiment"], test_size = .1, random_state=1)

#convert to 1d arrays
y_train, y_test = np.ravel(y_train), np.ravel(y_test)

#ensure our 1/0 values are integers for the pipeline model we will use
y_train, y_test = y_train.astype(int), y_test.astype(int)

#fit on ONLY the training data
tfidf.fit(X_train)

#transform both train and test data
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)

## 4.4 Sampling Techniques for Imbalance

### 4.4.1 Sampling overview

In [4]:
# 30 % of our reviews are negative (what we're trying to predict)
fashion.neg_sentiment.value_counts(normalize = True)

0    0.695061
1    0.304939
Name: neg_sentiment, dtype: float64



In data sets with an imbalanced split between the classes we are trying to predict, there are few possible approaches to try to improve the target metric our model (classifier) is optimizing for.

1. Over-sampling - if we train with a higher proportion of the class we are trying to predict using resampling, we may be able to improve the result for our classifier.

2. Under-sampling - by the same logic, we can under-sample the majority classes we are NOT trying to predict.

3. Synthesize data: ADASYN and SMOTE both use Nearest Neigbhors algorithms to generate artificial points that are located "close" in the n-dimensional feature space of the target class to the actual data points. Conceptually, we can think of it as if we gathered MORE data, and are assuming it looks similar to the current data we have. It will be unlikely to have strong outliers due to the nature of the algorithm and will be more "clumped" together than if we gathered more "real" data.


### 4.4.2 Timing comparison of samplers

In [5]:
# Initialize samplers

In [6]:
smote = SMOTE() #default 5 neighbors

ada = ADASYN() #default 5 neighbors

ros = RandomOverSampler()

rus = RandomUnderSampler()

Using a loop with increasing counts for the rows used, we can combine with the timeit cell magic to get a sense of how long each of our sampling methods takes.

In [7]:
samplers = {"smote":smote, "ada":ada, "ros":ros, "rus":rus}

timing_dict = {sampler:{k:None for k in range(1,25)} for sampler in samplers.keys()}

for sampler in samplers.keys():
    for i in range(1,25):
    
        n_rows = i*5000
    
        time_var = %timeit -n1 -o samplers[sampler].fit_resample(X_train[0:n_rows,:], y_train[0:n_rows])

        timing_dict[sampler][i] = np.mean(time_var.all_runs)

dfs = []

for sampler_type in samplers.keys():

    dfs.append(pd.DataFrame([(k,v) for k,v in timing_dict[sampler_type].items()], columns=["iter", "time_in_seconds"]))

for sampler_type, df in zip(samplers.keys(), dfs):
    df["sampler_type"] = sampler_type
combined_df = pd.concat(dfs, axis=0)

33.7 ms ± 3.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
143 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
317 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
574 ms ± 3.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
886 ms ± 2.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.29 s ± 2.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.73 s ± 5.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.13 s ± 2.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.71 s ± 23.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.51 s ± 4.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.1 s ± 5.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.85 s ± 5.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.87 s ± 5.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.83 s ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.79 s ± 10.6 ms per loop (mean ± std. dev. of 7 

In [8]:
px.line(data_frame=combined_df, x="iter", y="time_in_seconds", color='sampler_type', title="Time Complexity of Sampling Methods for Class Imbalance")

As shown above, the synthetic data generation algorithms (SMOTE and ADASYN) which rely on K-Nearest Neighbors algorithms to generate the new data, run slower and slower on larger proportions of our data.

The KNN worst case time complexity is O(N * kD) (where N = number of data points/row, K is the number of neighbors used, and D is the number of features/columns). Because we have vectorized the reviews with TFIDF using unigrams and bigrams, the width of the data is roughly 2 million columns.

## 4.5 Bayesian Hyperparameter Optimization with Optuna

### 4.5.1 About Optuna:

A python library that allows you to write custom functions for Bayesian Optimization. When searching over a range of models and hyperparameters, the time complexity of grid search is exponential due to the brute-force nature of the algorithm. This quickly becomes untenable with more than a few different parameters.

### 4.5.2 Custom Optimization Function

#### Funtion optimization params:

##### 1. Sampling:
  * Random over-sampling
  * Random under-sampling
  * Synthetic Minority Over-sampling ('smote')
  * Adaptive Synthetic Over-sampling ('adasyn')
  * None

##### 2. Models:
  * Logistic Regression ('logreg')
    - C (regularization parameter)
  * XGBoost ('xgboost')
    - learning rate
    - max depth
    - number of estimators
  * LightGBM ('lgbm')
    - learning rate
    - max depth
    - number of estimators


In [9]:
def objective(trial, sub_sample_prop, model_choices):
    """
    params:
        trial: functions like a self parameter for the optimization function)
        sub_sample_prop: amount of the entire train set to sample from for current trial
        model_choices: allow dynamic selection between three model types   
    """

    #sampler

    
    sampler_type = trial.suggest_categorical('sampler', [None, 'ros', 'rus', 'smote', 'ada'])

    if sampler_type == 'ros':
        sampler = RandomOverSampler(random_state=0)
    
    elif sampler_type == 'smote':
        k_neighbors = trial.suggest_int('k_neighbors', 2,10)
        sampler = SMOTE(random_state=0, k_neighbors=k_neighbors)
    
    elif sampler_type == 'rus':
        sampler = RandomUnderSampler(random_state=0)
    
    elif sampler_type == 'ada':
        n_neighbors = trial.suggest_int('n_neighbors', 2,10)
        sampler = ADASYN(n_neighbors=n_neighbors)
    
    else:
        sampler = None


    model_type = trial.suggest_categorical('classifier', model_choices)

    if model_type == 'logreg':
        #optimize params
        C = trial.suggest_categorical('C', [1.0, 0.1, 0.01]) #note: models with larger values for C failed to converge
        
        #model
        model = LogisticRegression(solver = "lbfgs", n_jobs=-1, max_iter=1000, C=C)

    elif model_type == 'xgboost':
        #optimize params
        learning_rate = trial.suggest_categorical('learning_rate', [0.2, 0.1, 0.01, .001, .0001])
        max_depth = trial.suggest_int('max_depth', 3, 20)
        n_estimators = trial.suggest_categorical('n_estimators', [200,500,1000, 2000, 4000])

        #model
        model = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, n_jobs=-1, verbosity=0, use_label_encoder=False)

    elif model_type == "lgbm":
        #optimize params
        learning_rate = trial.suggest_categorical('learning_rate', [0.2, 0.1, 0.01, .001, .0001])
        max_depth = trial.suggest_int('max_depth', 3, 20)
        n_estimators = trial.suggest_categorical('n_estimators', [200,500,1000, 2000,4000])

        #model
        model = lgb.LGBMClassifier(max_depth = max_depth, n_estimators=n_estimators)
    
    pipeline = Pipeline([('sampler', sampler), ('model',model)])
    
    X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=sub_sample_prop, random_state=trial.number)

    print("N_rows: ", X_train_sample.shape[0])

    pipeline.fit(X_train_sample, y_train_sample)
    
    #using the original X_test from the top train_test_split above
    y_preds = pipeline.predict(X_test)

    return recall_score(y_preds, y_test)

### 4.5.3 Run 1

We'll test out a loop with a very small proportion of our data to train on, to see if the different sampling methods give a better recall score. Because they run slowly (smote and adasyn especially), it would be nice to skip them to save computation time if they are not proving useful. With a small subset of the data to train on (fewer members of the positive class), it should be an ideal test case to see their effectiveness.

In [10]:
# we have to use a lambda function as a wrapper in order to pass in parameters to the optimize function

func = lambda trial: objective(trial, .002, ["xgboost", "lgbm", "logreg"])

study = optuna.create_study(direction='maximize')

study.optimize(func, n_trials=30)

[I 2022-08-09 00:21:11,343] A new study created in memory with name: no-name-aa814fad-e831-4393-b1c9-729565b956e3


N_rows:  1572


[I 2022-08-09 00:21:11,937] Trial 0 finished with value: 0.5460531432274789 and parameters: {'sampler': 'rus', 'classifier': 'lgbm', 'learning_rate': 0.001, 'max_depth': 4, 'n_estimators': 500}. Best is trial 0 with value: 0.5460531432274789.


N_rows:  1572


[I 2022-08-09 00:21:12,890] Trial 1 finished with value: 0.6867019019327958 and parameters: {'sampler': 'ros', 'classifier': 'lgbm', 'learning_rate': 0.001, 'max_depth': 4, 'n_estimators': 1000}. Best is trial 1 with value: 0.6867019019327958.


...

[I 2022-08-09 00:24:39,462] Trial 27 finished with value: 0.7028662279733251 and parameters: {'sampler': None, 'classifier': 'lgbm', 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200}. Best is trial 23 with value: 1.0.


N_rows:  1572


[I 2022-08-09 00:25:10,434] Trial 28 finished with value: 0.7176679895990147 and parameters: {'sampler': None, 'classifier': 'xgboost', 'learning_rate': 0.2, 'max_depth': 16, 'n_estimators': 2000}. Best is trial 23 with value: 1.0.


N_rows:  1572


[I 2022-08-09 00:25:10,944] Trial 29 finished with value: 0.875 and parameters: {'sampler': None, 'classifier': 'logreg', 'C': 0.1}. Best is trial 23 with value: 1.0.


#### Run 1 Results:

Training on a small portion of the data (~1500 rows), our best model was consistently the Logistic Regression and none of the sampling methods seemed to improve the recall of our model.

### 4.5.4 Run 2
With a more reasonable training size, we'll run a longer study and compare how well the different models and sampling methods work.

In [11]:
# we have to use a lambda function as a wrapper in order to pass in parameters to the optimize function

func = lambda trial: objective(trial, .1, ["xgboost", "lgbm", "logreg"])

study = optuna.create_study(direction='maximize')

study.optimize(func, n_trials=200)

[I 2022-08-09 00:25:10,986] A new study created in memory with name: no-name-d5ca5c8c-f9bf-4f72-8136-586d0d14ddd0


N_rows:  78601


[I 2022-08-09 00:26:22,830] Trial 0 finished with value: 0.8257125772373929 and parameters: {'sampler': None, 'classifier': 'lgbm', 'learning_rate': 0.1, 'max_depth': 12, 'n_estimators': 4000}. Best is trial 0 with value: 0.8257125772373929.


N_rows:  78601


[I 2022-08-09 00:26:32,640] Trial 1 finished with value: 0.7119376087496893 and parameters: {'sampler': 'smote', 'k_neighbors': 8, 'classifier': 'logreg', 'C': 0.1}. Best is trial 0 with value: 0.8257125772373929.


...

[I 2022-08-09 01:19:47,932] Trial 197 finished with value: 0.764406286379511 and parameters: {'sampler': None, 'classifier': 'xgboost', 'learning_rate': 0.001, 'max_depth': 16, 'n_estimators': 500}. Best is trial 37 with value: 0.9454374412041392.


N_rows:  78601


[I 2022-08-09 01:19:49,367] Trial 198 finished with value: 0.9444926279271466 and parameters: {'sampler': None, 'classifier': 'logreg', 'C': 0.01}. Best is trial 37 with value: 0.9454374412041392.


N_rows:  78601


[I 2022-08-09 01:19:50,635] Trial 199 finished with value: 0.9417249417249417 and parameters: {'sampler': None, 'classifier': 'logreg', 'C': 0.01}. Best is trial 37 with value: 0.9454374412041392.


### 4.5.5 Run 3

For the final run, we'll use a larger training size, but will skip the sampling methods completely.

In [12]:
def objective(trial, sub_sample_prop, model_choices):
    """
    params:
        trial: functions like a self parameter for the optimization function)
        sub_sample_prop: amount of the entire train set to sample from for current trial
        model_choices: allow dynamic selection between three model types   
    """

    #sampler

    sampler = None


    model_type = trial.suggest_categorical('classifier', model_choices)

    if model_type == 'logreg':
        #optimize params
        C = trial.suggest_categorical('C', [1.0, 0.1, 0.01]) #note: models with larger values for C failed to converge
        
        #model
        model = LogisticRegression(solver = "lbfgs", n_jobs=-1, max_iter=1000, C=C)

    elif model_type == 'xgboost':
        #optimize params
        learning_rate = trial.suggest_categorical('learning_rate', [0.2, 0.1, 0.01, .001, .0001])
        max_depth = trial.suggest_int('max_depth', 3, 20)
        n_estimators = trial.suggest_categorical('n_estimators', [200,500,1000, 2000, 4000])

        #model
        model = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, n_jobs=-1, verbosity=0, use_label_encoder=False)

    elif model_type == "lgbm":
        #optimize params
        learning_rate = trial.suggest_categorical('learning_rate', [0.2, 0.1, 0.01, .001, .0001])
        max_depth = trial.suggest_int('max_depth', 3, 20)
        n_estimators = trial.suggest_categorical('n_estimators', [200,500,1000, 2000,4000])

        #model
        model = lgb.LGBMClassifier(max_depth = max_depth, n_estimators=n_estimators)
    
    pipeline = Pipeline([('sampler', sampler), ('model',model)])
    
    X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=sub_sample_prop, random_state=trial.number)

    print("N_rows: ", X_train_sample.shape[0])

    pipeline.fit(X_train_sample, y_train_sample)
    
    #using the original X_test from the top train_test_split above
    y_preds = pipeline.predict(X_test)

    return recall_score(y_preds, y_test)

In [13]:
# We've seen above that the under/over sampling is not effective, even on small training sets. Here we will skip it, but use a larger proportion of our training data to tune hyperparameters

func = lambda trial: objective(trial, .2, ["xgboost", "lgbm", "logreg"])

study = optuna.create_study(direction='maximize')

study.optimize(func, n_trials=200)

[I 2022-08-09 01:19:50,708] A new study created in memory with name: no-name-ed45df43-167f-474e-aa48-9ede165c2302


N_rows:  157203


[I 2022-08-09 01:19:54,369] Trial 0 finished with value: 0.8552049298978452 and parameters: {'classifier': 'logreg', 'C': 1.0}. Best is trial 0 with value: 0.8552049298978452.


N_rows:  157203


[I 2022-08-09 01:22:41,388] Trial 1 finished with value: 0.7556986866602928 and parameters: {'classifier': 'xgboost', 'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 1000}. Best is trial 0 with value: 0.8552049298978452.


...

[I 2022-08-09 02:55:18,225] Trial 197 finished with value: 0.914272527979693 and parameters: {'classifier': 'logreg', 'C': 0.01}. Best is trial 35 with value: 0.9164345403899722.


N_rows:  157203


[I 2022-08-09 02:55:20,051] Trial 198 finished with value: 0.9124719101123595 and parameters: {'classifier': 'logreg', 'C': 0.01}. Best is trial 35 with value: 0.9164345403899722.


N_rows:  157203


[I 2022-08-09 02:55:21,722] Trial 199 finished with value: 0.9126366813211588 and parameters: {'classifier': 'logreg', 'C': 0.01}. Best is trial 35 with value: 0.9164345403899722.


In [14]:
study_df = study.trials_dataframe()

## 4.6 Modeling Summary

1. Train size of 18% of the total data

2. Validate on 10% of the total dataset

3. Using Bayesian optimization with optuna

### 4.6.1 Model Comparisons

### XGBoost results:

1. The best recall for any xgboost models was around 0.85

2. The XGBoost was the slowest to train and predict of the 3 models, with an average time of 5 minutes and 28 seconds.

3. The top XGBoost models all had similar max depths of around 10-20 and varying estimators between 500 and 4000.

In [15]:
best_xg_model = study_df[study_df["params_classifier"] == "xgboost"].sort_values("value", ascending=False).head(1)
print(best_xg_model)
best_xg_model.to_csv("best_models.csv", index=False, mode="w")

    number     value             datetime_start          datetime_complete  \
97      97  0.841939 2022-08-09 02:35:41.225839 2022-08-09 02:37:34.537309   

                 duration  params_C params_classifier  params_learning_rate  \
97 0 days 00:01:53.311470       NaN           xgboost                   0.1   

    params_max_depth  params_n_estimators     state  
97              17.0                500.0  COMPLETE  


In [16]:
study_df[study_df["params_classifier"] == "xgboost"].describe()

,number,value,duration,params_C,params_learning_rate,params_max_depth,params_n_estimators
count,14.000000,14.000000,14,0.0,14.000000,14.000000,14.000000
mean,59.000000,0.790180,0 days 00:05:28.169958,NaN,0.051871,13.142857,1414.285714
std,53.864502,0.037802,0 days 00:07:58.468925269,NaN,0.084370,3.613163,1484.720716
min,1.000000,0.749762,0 days 00:00:29.058107,NaN,0.000100,6.000000,200.000000
25%,12.750000,0.759043,0 days 00:01:11.018197,NaN,0.001000,10.250000,275.000000
50%,43.000000,0.771867,0 days 00:02:44.950635500,NaN,0.001000,14.000000,1000.000000
75%,92.500000,0.833926,0 days 00:03:15.305097,NaN,0.077500,16.000000,1750.000000
max,168.000000,0.841939,0 days 00:27:57.931114,NaN,0.200000,19.000000,4000.000000


In [17]:
study_df[study_df["params_classifier"] == "xgboost"].sort_values("value", ascending=False).head(5)

,number,value,datetime_start,datetime_complete,duration,params_C,params_classifier,params_learning_rate,params_max_depth,params_n_estimators,state
97,97,0.841939,2022-08-09 02:35:41.225839,2022-08-09 02:37:34.537309,0 days 00:01:53.311470,NaN,xgboost,0.10,17.0,500.0,COMPLETE
7,7,0.840946,2022-08-09 01:28:07.298355,2022-08-09 01:31:26.477149,0 days 00:03:19.178794,NaN,xgboost,0.20,16.0,1000.0,COMPLETE
27,27,0.840291,2022-08-09 01:35:24.936693,2022-08-09 01:36:53.408932,0 days 00:01:28.472239,NaN,xgboost,0.20,14.0,500.0,COMPLETE
118,118,0.834855,2022-08-09 02:39:02.922890,2022-08-09 02:47:49.002882,0 days 00:08:46.079992,NaN,xgboost,0.20,11.0,4000.0,COMPLETE
8,8,0.831136,2022-08-09 01:31:26.477985,2022-08-09 01:34:30.161991,0 days 00:03:03.684006,NaN,xgboost,0.01,12.0,1000.0,COMPLETE


### LightGBM Reults:

1. With the size of the data (roughly 800,000 reviews/rows, 2 million features/columns), the LightGB model only took an average of 58 seconds to train/predict.
This was much faster than the XGBoost, and it is often used for large datasets instead.

2. The best LightGBM models all had similar recall scores to the best XGboost model, around 0.85 as well.

3. LightGBM models did well with lower learning rates and higher depth or a higher learning rate and relatively lower depth.

In [18]:
best_lgb_model = study_df[study_df["params_classifier"] == "lgbm"].sort_values("value", ascending=False).head(1)
print(best_lgb_model)
best_lgb_model.to_csv("best_models.csv", index=False, mode="a")

     number     value             datetime_start          datetime_complete  \
106     106  0.844527 2022-08-09 02:37:51.645079 2022-08-09 02:38:43.209772   

                  duration  params_C params_classifier  params_learning_rate  \
106 0 days 00:00:51.564693       NaN              lgbm                  0.01   

     params_max_depth  params_n_estimators     state  
106              20.0               1000.0  COMPLETE  


In [19]:
study_df[study_df["params_classifier"] == "lgbm"].describe()

,number,value,duration,params_C,params_learning_rate,params_max_depth,params_n_estimators
count,14.000000,14.000000,14,0.0,14.000000,14.000000,14.000000
mean,65.214286,0.835981,0 days 00:00:57.888010928,NaN,0.074307,9.357143,1457.142857
std,57.180118,0.011132,0 days 00:00:51.526948337,NaN,0.079951,6.159510,1281.654151
min,3.000000,0.807192,0 days 00:00:10.332197,NaN,0.000100,3.000000,200.000000
25%,21.250000,0.833544,0 days 00:00:19.953306250,NaN,0.010000,4.250000,500.000000
50%,48.000000,0.839853,0 days 00:00:50.945442,NaN,0.055000,7.500000,1000.000000
75%,101.250000,0.844032,0 days 00:01:10.435751,NaN,0.100000,12.500000,2000.000000
max,180.000000,0.844527,0 days 00:02:57.835429,NaN,0.200000,20.000000,4000.000000


In [20]:
study_df[study_df["params_classifier"] == "lgbm"].sort_values("value", ascending=False).head(5)

,number,value,datetime_start,datetime_complete,duration,params_C,params_classifier,params_learning_rate,params_max_depth,params_n_estimators,state
106,106,0.844527,2022-08-09 02:37:51.645079,2022-08-09 02:38:43.209772,0 days 00:00:51.564693,NaN,lgbm,0.0100,20.0,1000.0,COMPLETE
3,3,0.844389,2022-08-09 01:23:10.448339,2022-08-09 01:24:17.147875,0 days 00:01:06.699536,NaN,lgbm,0.0100,7.0,2000.0,COMPLETE
6,6,0.844384,2022-08-09 01:27:16.971263,2022-08-09 01:28:07.297454,0 days 00:00:50.326191,NaN,lgbm,0.0001,18.0,1000.0,COMPLETE
56,56,0.844316,2022-08-09 02:12:41.510831,2022-08-09 02:14:00.486834,0 days 00:01:18.976003,NaN,lgbm,0.2000,11.0,2000.0,COMPLETE
70,70,0.843180,2022-08-09 02:15:25.598517,2022-08-09 02:16:18.045150,0 days 00:00:52.446633,NaN,lgbm,0.2000,4.0,2000.0,COMPLETE


### Logistic Regression Results:

1. Train/predict was the fastest of the 3 models. It is a simpler model, using gradient descent, which computes very quickly. On average, it only took 2 seconds.

2. The best linear regression models had a C value (determining gradient descent speed) of 0.01 and a recall near 0.91.

3. Overall, despite being the simplest model, the Logistic Regression model gave the best results, as well as training/predicting far, far quicker.

In [21]:
best_lr_model = study_df[study_df["params_classifier"] == "logreg"].sort_values("value", ascending=False).head(1)
print(best_lr_model)
best_lr_model.to_csv("best_models.csv", index=False, mode="a")

    number     value             datetime_start          datetime_complete  \
35      35  0.916435 2022-08-09 01:40:03.147489 2022-08-09 01:40:04.929709   

                 duration  params_C params_classifier  params_learning_rate  \
35 0 days 00:00:01.782220      0.01            logreg                   NaN   

    params_max_depth  params_n_estimators     state  
35               NaN                  NaN  COMPLETE  


In [22]:
study_df[study_df["params_classifier"] == "logreg"].describe()

,number,value,duration,params_C,params_learning_rate,params_max_depth,params_n_estimators
count,172.000000,172.000000,172,172.000000,0.0,0.0,0.0
mean,105.587209,0.906174,0 days 00:00:01.895718837,0.078547,NaN,NaN,NaN
std,56.228112,0.018383,0 days 00:00:00.879439925,0.242477,NaN,NaN,NaN
min,0.000000,0.852294,0 days 00:00:01.425250,0.010000,NaN,NaN,NaN
25%,58.750000,0.912021,0 days 00:00:01.511956750,0.010000,NaN,NaN,NaN
50%,107.500000,0.913222,0 days 00:00:01.685848500,0.010000,NaN,NaN,NaN
75%,154.250000,0.913836,0 days 00:00:01.716180500,0.010000,NaN,NaN,NaN
max,199.000000,0.916435,0 days 00:00:05.955641,1.000000,NaN,NaN,NaN


In [23]:
study_df[study_df["params_classifier"] == "logreg"].sort_values("value", ascending=False).head(5)

,number,value,datetime_start,datetime_complete,duration,params_C,params_classifier,params_learning_rate,params_max_depth,params_n_estimators,state
35,35,0.916435,2022-08-09 01:40:03.147489,2022-08-09 01:40:04.929709,0 days 00:00:01.782220,0.01,logreg,NaN,NaN,NaN,COMPLETE
161,161,0.915943,2022-08-09 02:51:16.061808,2022-08-09 02:51:17.750527,0 days 00:00:01.688719,0.01,logreg,NaN,NaN,NaN,COMPLETE
185,185,0.915468,2022-08-09 02:54:56.144753,2022-08-09 02:54:57.855027,0 days 00:00:01.710274,0.01,logreg,NaN,NaN,NaN,COMPLETE
91,91,0.915353,2022-08-09 02:35:31.325249,2022-08-09 02:35:32.803748,0 days 00:00:01.478499,0.01,logreg,NaN,NaN,NaN,COMPLETE
136,136,0.915318,2022-08-09 02:49:29.429322,2022-08-09 02:49:31.132673,0 days 00:00:01.703351,0.01,logreg,NaN,NaN,NaN,COMPLETE


### 4.6.2 Conclusion

1. Though quite long to process, we examined the effects of different sampling methods to help with the class imbalance in our notebook.

2. We found no benefit using resampling/synthetic data generation, even on very small training sets, with few instances of the positive (predicted) class.

3. Our best model, both in terms of recall and prediction time, was the Logistic Regression model.

4. In the final notebook, we will look at the interpretability of the three models and compare any similarities and differences.